# Speed dating experiment
The goal of this project is to extract knowledge from a database created during a speed dating experiment in the US. After cleaning data we will first extract simple facts. We will then focus on the question of shared interest and their influence on getting a second date. 

## 1. Exploring and cleaning

Import data and libs

In [1]:
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import plotly.io as pio

pio.renderers.default = "svg"

# If you are on the workspaces:
pio.renderers.default = "iframe_connected"

import plotly.express as px
import plotly.graph_objects as go

df = pd.read_csv('Speed Dating Data.csv', encoding="ISO-8859-1")

#### Visualise a sample of raw data. 
Random sample gives a little more information than the first rows since some columns can be ordered. To fully understand the data it is better to read the columns description given by the author.

In [2]:
# Add more visualisable columns 
pd.options.display.max_columns = 200
display(df.sample(10))

,iid,id,gender,idg,condtn,wave,round,position,positin1,order,partner,pid,match,int_corr,samerace,age_o,race_o,pf_o_att,pf_o_sin,pf_o_int,pf_o_fun,pf_o_amb,pf_o_sha,dec_o,attr_o,sinc_o,intel_o,fun_o,amb_o,shar_o,like_o,prob_o,met_o,age,field,field_cd,undergra,mn_sat,tuition,race,imprace,imprelig,from,zipcode,income,goal,date,go_out,career,career_c,sports,tvsports,exercise,dining,museums,art,hiking,gaming,clubbing,reading,tv,theater,movies,concerts,music,shopping,yoga,exphappy,expnum,attr1_1,sinc1_1,intel1_1,fun1_1,amb1_1,shar1_1,attr4_1,sinc4_1,intel4_1,fun4_1,amb4_1,shar4_1,attr2_1,sinc2_1,intel2_1,fun2_1,amb2_1,shar2_1,attr3_1,sinc3_1,fun3_1,intel3_1,amb3_1,attr5_1,sinc5_1,intel5_1,fun5_1,amb5_1,dec,attr,sinc,intel,fun,amb,shar,like,prob,met,match_es,attr1_s,sinc1_s,intel1_s,fun1_s,amb1_s,shar1_s,attr3_s,sinc3_s,intel3_s,fun3_s,amb3_s,satis_2,length,numdat_2,attr7_2,sinc7_2,intel7_2,fun7_2,amb7_2,shar7_2,attr1_2,sinc1_2,intel1_2,fun1_2,amb1_2,shar1_2,attr4_2,sinc4_2,intel4_2,fun4_2,amb4_2,shar4_2,attr2_2,sinc2_2,intel2_2,fun2_2,amb2_2,shar2_2,attr3_2,sinc3_2,intel3_2,fun3_2,amb3_2,attr5_2,sinc5_2,intel5_2,fun5_2,amb5_2,you_call,them_cal,date_3,numdat_3,num_in_3,attr1_3,sinc1_3,intel1_3,fun1_3,amb1_3,shar1_3,attr7_3,sinc7_3,intel7_3,fun7_3,amb7_3,shar7_3,attr4_3,sinc4_3,intel4_3,fun4_3,amb4_3,shar4_3,attr2_3,sinc2_3,intel2_3,fun2_3,amb2_3,shar2_3,attr3_3,sinc3_3,intel3_3,fun3_3,amb3_3,attr5_3,sinc5_3,intel5_3,fun5_3,amb5_3
6353,411,15.0,1,30,2,15,18,1,1.0,1,16,394.0,0,0.40,1,24.0,2.0,18.0,19.0,16.0,15.0,15.0,17.0,0,9.5,7.0,7.0,8.0,9.0,6.0,7.0,6.0,2.0,34.0,Finance,8.0,The American University,"1,206.00","22,481.00",2.0,8.0,2.0,Washington DC,"20,817","80,006.00",3.0,4.0,1.0,M&A Advisory,7.0,7.0,6.0,6.0,7.0,7.0,6.0,3.0,3.0,1.0,8.0,1.0,7.0,6.0,7.0,10.0,8.0,3.0,3.0,NaN,35.0,15.0,15.0,25.0,10.0,0.0,40.0,20.0,15.0,15.0,5.0,5.0,60.0,0.0,10.0,20.0,0.0,0.0,8.0,6.0,9.0,7.0,5.0,7.0,7.0,7.0,8.0,6.0,0,6.0,9.0,8.0,7.0,4.0,5.0,6.0,7.0,2.0,2.0,80.0,0.0,0.0,20.0,0.0,0.0,9.0,7.0,8.0,10.0,6.0,8.0,1.0,3.0,NaN,NaN,NaN,NaN,NaN,NaN,50.0,0.0,25.0,25.0,0.0,0.0,40.0,20.0,20.0,10.0,10.0,0.0,70.0,20.0,0.0,10.0,0.0,0.0,8.0,7.0,9.0,10.0,7.0,8.0,6.0,8.0,8.0,6.0,1.0,1.0,0.0,0.0,NaN,80.0,0.0,10.0,10.0,0.0,0.0,70.0,0.0,10.0,20.0,0.0,0.0,70.0,0.0,15.0,15.0,0.0,0.0,80.0,20.0,0.0,0.0,0.0,0.0,7.0,8.0,8.0,9.0,6.0,7.0,7.0,8.0,9.0,7.0
4271,285,13.0,1,26,2,11,21,8,8.0,5,9,260.0,1,-0.18,0,27.0,4.0,19.0,20.0,19.0,14.0,13.0,15.0,1,6.0,9.0,7.0,7.0,6.0,5.0,7.0,4.0,2.0,24.0,engineering,5.0,tech school,NaN,NaN,6.0,1.0,1.0,International Student,"10,025",NaN,4.0,6.0,6.0,finance or engineering,7.0,5.0,7.0,6.0,10.0,9.0,10.0,9.0,14.0,5.0,9.0,7.0,3.0,10.0,10.0,10.0,10.0,10.0,10.0,NaN,20.0,20.0,25.0,25.0,5.0,5.0,20.0,20.0,25.0,25.0,5.0,5.0,20.0,20.0,25.0,25.0,5.0,5.0,8.0,10.0,10.0,10.0,8.0,8.0,10.0,10.0,8.0,10.0,1,6.0,6.0,6.0,6.0,6.0,6.0,6.0,6.0,0.0,5.0,30.0,20.0,10.0,10.0,20.0,10.0,9.0,9.0,9.0,9.0,9.0,7.0,1.0,2.0,NaN,NaN,NaN,NaN,NaN,NaN,30.0,10.0,20.0,20.0,10.0,10.0,30.0,10.0,20.0,20.0,10.0,10.0,30.0,10.0,20.0,20.0,10.0,10.0,9.0,9.0,9.0,9.0,9.0,8.0,8.0,8.0,8.0,8.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
815,56,1.0,0,1,1,3,10,5,NaN,5,8,73.0,0,-0.31,1,22.0,2.0,30.0,10.0,20.0,30.0,0.0,10.0,0,7.0,7.0,6.0,5.0,4.0,5.0,5.0,5.0,2.0,23.0,social work,11.0,NaN,NaN,NaN,2.0,10.0,9.0,Florida,"33,496","41,466.00",1.0,3.0,1.0,social worker,9.0,2.0,1.0,10.0,10.0,5.0,5.0,2.0,1.0,7.0,6.0,8.0,8.0,7.0,6.0,9.0,10.0,4.0,3.0,15.0,17.0,18.0,18.0,15.0,17.0,15.0,NaN,NaN,NaN,NaN,NaN,NaN,32.0,10.0,12.0,27.0,7.0,12.0,10.0,10.0,9.0,8.0,5.0,NaN,NaN,NaN,NaN,NaN,1,4.0,10.0,9.0,8.0,9.0,7.0,6.0,8.0,2.0,3.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.0,3.0,3.0,NaN,NaN,NaN,NaN,NaN,NaN,20.0,15.0,15.0,15.0,20.0,15.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,9.0,9.0,7.0,9.0,7.0,NaN,NaN,NaN,NaN,NaN,0.0,2.0,0.0,NaN,NaN,15.0,18.0,18.0,17.0,18.0,14.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN

#### Check missing values
We will drop columns with too many missing values. Let's define a threshold: we drop columns with more than 75% missing values. As we wont have enough time to explore the whole dataset, this is a first selection we have to make. We could explore later the dropped columns if needed.

In [3]:
# Calculating missing values for every column
missing_values = df.isnull().sum()
missing_values = missing_values/len(df)*100

# Sorting table
missing_values = missing_values.sort_values(ascending=False)

# Print missing values percentage
with pd.option_context('display.max_rows', None):
    print(missing_values)

num_in_3    92.026737
numdat_3    82.143710
expnum      78.515159
sinc7_2     76.665075
amb7_2      76.665075
shar7_2     76.438291
attr7_2     76.318931
intel7_2    76.318931
fun7_2      76.318931
amb5_3      75.936978
attr7_3     75.936978
sinc7_3     75.936978
intel7_3    75.936978
fun7_3      75.936978
amb7_3      75.936978
shar7_3     75.936978
shar2_3     75.936978
attr5_3     75.936978
sinc5_3     75.936978
intel5_3    75.936978
fun5_3      75.936978
attr4_3     64.681308
sinc4_3     64.681308
intel4_3    64.681308
fun4_3      64.681308
amb4_3      64.681308
shar4_3     64.681308
attr2_3     64.681308
sinc2_3     64.681308
fun2_3      64.681308
intel2_3    64.681308
amb2_3      64.681308
mn_sat      62.604440
tuition     57.233230
you_call    52.566245
shar1_3     52.566245
date_3      52.566245
attr1_3     52.566245
sinc1_3     52.566245
intel1_3    52.566245
fun1_3      52.566245
amb1_3      52.566245
attr3_3     52.566245
sinc3_3     52.566245
intel3_3    52.566245
fun3_3    

In [4]:
# Create a new dataframe keeping only columns with less than 60% missing values
dating = df.iloc[:,:]
for col_name, col_missing_values in zip(missing_values.index, missing_values):
    if col_missing_values >= 60: dating = dating.drop(col_name, axis=1)

In [5]:
dating.head(3)

,iid,id,gender,idg,condtn,wave,round,position,positin1,order,partner,pid,match,int_corr,samerace,age_o,race_o,pf_o_att,pf_o_sin,pf_o_int,pf_o_fun,pf_o_amb,pf_o_sha,dec_o,attr_o,sinc_o,intel_o,fun_o,amb_o,shar_o,like_o,prob_o,met_o,age,field,field_cd,undergra,tuition,race,imprace,imprelig,from,zipcode,income,goal,date,go_out,career,career_c,sports,tvsports,exercise,dining,museums,art,hiking,gaming,clubbing,reading,tv,theater,movies,concerts,music,shopping,yoga,exphappy,attr1_1,sinc1_1,intel1_1,fun1_1,amb1_1,shar1_1,attr4_1,sinc4_1,intel4_1,fun4_1,amb4_1,shar4_1,attr2_1,sinc2_1,intel2_1,fun2_1,amb2_1,shar2_1,attr3_1,sinc3_1,fun3_1,intel3_1,amb3_1,attr5_1,sinc5_1,intel5_1,fun5_1,amb5_1,dec,attr,sinc,intel,fun,amb,shar,like,prob,met,match_es,attr1_s,sinc1_s,intel1_s,fun1_s,amb1_s,shar1_s,attr3_s,sinc3_s,intel3_s,fun3_s,amb3_s,satis_2,length,numdat_2,attr1_2,sinc1_2,intel1_2,fun1_2,amb1_2,shar1_2,attr4_2,sinc4_2,intel4_2,fun4_2,amb4_2,shar4_2,attr2_2,sinc2_2,intel2_2,fun2_2,amb2_2,shar2_2,attr3_2,sinc3_2,intel3_2,fun3_2,amb3_2,attr5_2,sinc5_2,intel5_2,fun5_2,amb5_2,you_call,them_cal,date_3,attr1_3,sinc1_3,intel1_3,fun1_3,amb1_3,shar1_3,attr3_3,sinc3_3,intel3_3,fun3_3,amb3_3
0,1,1.0,0,1,1,1,10,7,NaN,4,1,11.0,0,0.14,0,27.0,2.0,35.0,20.0,20.0,20.0,0.0,5.0,0,6.0,8.0,8.0,8.0,8.0,6.0,7.0,4.0,2.0,21.0,Law,1.0,NaN,NaN,4.0,2.0,4.0,Chicago,"60,521","69,487.00",2.0,7.0,1.0,lawyer,NaN,9.0,2.0,8.0,9.0,1.0,1.0,5.0,1.0,5.0,6.0,9.0,1.0,10.0,10.0,9.0,8.0,1.0,3.0,15.0,20.0,20.0,15.0,15.0,15.0,NaN,NaN,NaN,NaN,NaN,NaN,35.0,20.0,15.0,20.0,5.0,5.0,6.0,8.0,8.0,8.0,7.0,NaN,NaN,NaN,NaN,NaN,1,6.0,9.0,7.0,7.0,6.0,5.0,7.0,6.0,2.0,4.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,6.0,2.0,1.0,19.44,16.67,13.89,22.22,11.11,16.67,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,6.0,7.0,8.0,7.0,6.0,NaN,NaN,NaN,NaN,NaN,1.0,1.0,0.0,15.0,20.0,20.0,15.0,15.0,15.0,5.0,7.0,7.0,7.0,7.0
1,1,1.0,0,1,1,1,10,7,NaN,3,2,12.0,0,0.54,0,22.0,2.0,60.0,0.0,0.0,40.0,0.0,0.0,0,7.0,8.0,10.0,7.0,7.0,5.0,8.0,4.0,2.0,21.0,Law,1.0,NaN,NaN,4.0,2.0,4.0,Chicago,"60,521","69,487.00",2.0,7.0,1.0,lawyer,NaN,9.0,2.0,8.0,9.0,1.0,1.0,5.0,1.0,5.0,6.0,9.0,1.0,10.0,10.0,9.0,8.0,1.0,3.0,15.0,20.0,20.0,15.0,15.0,15.0,NaN,NaN,NaN,NaN,NaN,NaN,35.0,20.0,15.0,20.0,5.0,5.0,6.0,8.0,8.0,8.0,7.0,NaN,NaN,NaN,NaN,NaN,1,7.0,8.0,7.0,8.0,5.0,6.0,7.0,5.0,1.0,4.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,6.0,2.0,1.0,19.44,16.67,13.89,22.22,11.11,16.67,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,6.0,7.0,8.0,7.0,6.0,NaN,NaN,NaN,NaN,NaN,1.0,1.0,0.0,15.0,20.0,20.0,15.0,15.0,15.0,5.0,7.0,7.0,7.0,7.0
2,1,1.0,0,1,1,1,10,7,NaN,10,3,13.0,1,0.16,1,22.0,4.0,19.0,18.0,19.0,18.0,14.0,12.0,1,10.0,10.0,10.0,10.0,10.0,10.0,10.0,10.0,1.0,21.0,Law,1.0,NaN,NaN,4.0,2.0,4.0,Chicago,"60,521","69,487.00",2.0,7.0,1.0,lawyer,NaN,9.0,2.0,8.0,9.0,1.0,1.0,5.0,1.0,5.0,6.0,9.0,1.0,10.0,10.0,9.0,8.0,1.0,3.0,15.0,20.0,20.0,15.0,15.0,15.0,NaN,NaN,NaN,NaN,NaN,NaN,35.0,20.0,15.0,20.0,5.0,5.0,6.0,8.0,8.0,8.0,7.0,NaN,NaN,NaN,NaN,NaN,1,5.0,8.0,9.0,8.0,5.0,7.0,7.0,NaN,1.0,4.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,6.0,2.0,1.0,19.44,16.67,13.89,22.22,11.11,16.67,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,6.0,7.0,8.0,7.0,6.0,NaN,NaN,NaN,NaN,NaN,1.0,1.0,0.0,15.0,20.0,20.0,15.0,15.0,15.0,5.0,7.0,7.0,7.0,7.0


#### Check data distribution
We rapidly check if we can find columns with remarkable distribution (too many or too few different values, extremly unbalanced distribution...). Everything seems ok.

In [6]:
# Describe the dataset
dating.describe(include="all")

,iid,id,gender,idg,condtn,wave,round,position,positin1,order,partner,pid,match,int_corr,samerace,age_o,race_o,pf_o_att,pf_o_sin,pf_o_int,pf_o_fun,pf_o_amb,pf_o_sha,dec_o,attr_o,sinc_o,intel_o,fun_o,amb_o,shar_o,like_o,prob_o,met_o,age,field,field_cd,undergra,tuition,race,imprace,imprelig,from,zipcode,income,goal,date,go_out,career,career_c,sports,tvsports,exercise,dining,museums,art,hiking,gaming,clubbing,reading,tv,theater,movies,concerts,music,shopping,yoga,exphappy,attr1_1,sinc1_1,intel1_1,fun1_1,amb1_1,shar1_1,attr4_1,sinc4_1,intel4_1,fun4_1,amb4_1,shar4_1,attr2_1,sinc2_1,intel2_1,fun2_1,amb2_1,shar2_1,attr3_1,sinc3_1,fun3_1,intel3_1,amb3_1,attr5_1,sinc5_1,intel5_1,fun5_1,amb5_1,dec,attr,sinc,intel,fun,amb,shar,like,prob,met,match_es,attr1_s,sinc1_s,intel1_s,fun1_s,amb1_s,shar1_s,attr3_s,sinc3_s,intel3_s,fun3_s,amb3_s,satis_2,length,numdat_2,attr1_2,sinc1_2,intel1_2,fun1_2,amb1_2,shar1_2,attr4_2,sinc4_2,intel4_2,fun4_2,amb4_2,shar4_2,attr2_2,sinc2_2,intel2_2,fun2_2,amb2_2,shar2_2,attr3_2,sinc3_2,intel3_2,fun3_2,amb3_2,attr5_2,sinc5_2,intel5_2,fun5_2,amb5_2,you_call,them_cal,date_3,attr1_3,sinc1_3,intel1_3,fun1_3,amb1_3,shar1_3,attr3_3,sinc3_3,intel3_3,fun3_3,amb3_3
count,8378.000000,8377.000000,8378.000000,8378.000000,8378.000000,8378.000000,8378.000000,8378.000000,6532.000000,8378.000000,8378.000000,8368.000000,8378.000000,8220.000000,8378.000000,8274.000000,8305.000000,8289.000000,8289.000000,8289.000000,8280.000000,8271.000000,8249.000000,8378.000000,8166.000000,8091.000000,8072.000000,8018.000000,7656.000000,7302.000000,8128.000000,8060.000000,7993.000000,8283.000000,8315,8296.000000,4914,3583,8315.000000,8299.000000,8299.000000,8299,7314,4279,8299.000000,8281.000000,8299.000000,8289,8240.000000,8299.000000,8299.000000,8299.000000,8299.000000,8299.000000,8299.000000,8299.000000,8299.000000,8299.000000,8299.000000,8299.000000,8299.000000,8299.000000,8299.000000,8299.000000,8299.000000,8299.000000,8277.000000,8299.000000,8299.000000,8299.000000,8289.000000,8279.000000,8257.000000,6489.000000,6489.000000,6489.000000,6489.000000,6489.000000,6467.000000,8299.000000,8299.000000,8299.000000,8299.000000,8289.000000,8289.000000,8273.000000,8273.000000,8273.000000,8273.000000,8273.000000,4906.000000,4906.000000,4906.000000,4906.000000,4906.000000,8378.000000,8176.000000,8101.000000,8082.000000,8028.000000,7666.000000,7311.000000,8138.000000,8069.000000,8003.000000,7205.000000,4096.000000,4096.000000,4096.000000,4096.000000,4096.000000,4096.000000,4000.00000,4000.000000,4000.000000,4000.000000,4000.000000,7463.000000,7463.000000,7433.000000,7445.000000,7463.000000,7463.000000,7463.000000,7463.000000,7463.000000,5775.000000,5775.000000,5775.000000,5775.000000,5775.000000,5775.000000,5775.000000,5775.00000,5775.000000,5775.000000,5775.000000,5775.000000,7463.000000,7463.000000,7463.000000,7463.000000,7463.000000,4377.000000,4377.000000,4377.000000,4377.000000,4377.000000,3974.000000,3974.000000,3974.000000,3974.000000,3974.000000,3974.000000,3974.000000,3974.000000,3974.000000,3974.000000,3974.000000,3974.000000,3974.000000,3974.000000
unique,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,259,NaN,241,115,NaN,NaN,NaN,269,409,261,NaN,NaN,NaN,367,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
top,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Business,NaN,UC Berkeley,"26,908.00",NaN,NaN,NaN,New York,0,"55,080.00",NaN,NaN,NaN,Finance,NaN,NaN,NaN,NaN,NaN,NaN,Na

In [7]:
# Count number of men and women
dating.groupby("gender").count()["iid"]

gender
0    4184
1    4194
Name: iid, dtype: int64

We can note that people participating have a average age of 26 and most of them are between 18 and 28. There is nearly the same number of men and women.

## 2. What are men and women looking for

### Define what men and women are mostly waiting from their partner.

In [8]:
# "We want to know what you look for in the opposite sex BEFORE EVENT"
#     wave 6-9: rating range 1-10 -> written in doc but appears to be false
#     wave 1-5 and 10-21: rate sum must be 100
expectations_index = [
    "attr1_1",
    "sinc1_1",
    "intel1_1",
    "fun1_1",
    "amb1_1",
    "shar1_1"
]

# Visual name of previous indexes
expectations_name = [
    "Attractive",
    "Sincere",
    "Intelligent",
    "Fun",
    "Ambitious",
    "Interests"
]

From now on we will focus on expectaions. We exclude waves 6 to 9 since they are different from the rest of the data.

In [9]:
# Exclude waves 6, 7, 8 and 9
dating = dating[(dating["wave"] <=5) | (dating["wave"] > 9)]

# Creating expectations tables for men and woment
expectations = dating[["gender", *expectations_index]]
female_expectations = expectations[expectations["gender"] == 0].mean()[expectations_index]
male_expectations = expectations[expectations["gender"] == 1].mean()[expectations_index]

In [10]:
fig = go.Figure()

# Spider plot with women expectations in men
fig.add_trace(go.Scatterpolar(
      r=male_expectations.to_list(),
      theta=expectations_name,
      fill='toself',
      name='Male expectations'
))
# Spider plot with men expectations in women
fig.add_trace(go.Scatterpolar(
      r=female_expectations.to_list(),
      theta=expectations_name,
      fill='toself',
      name='Female expectations'
))

# Create layout
fig.update_layout(
  polar=dict(
    radialaxis=dict(
      visible=True,
      range=[0, 31]
    )),
  showlegend=True,
  title={"text":"People expectations for their ideal partner"}
)

fig.show()

We can see that
- men value attractiveness 
- women value balanced personalities

### Are people consistent?
The previous graph shows expectations people think they have. But do they actually say "Yes" to people they think they would? <br/>
To give a first answer to this question let's compare people expectations with the rating they gave to the other person on the same criterias. We define the affinity score to do so.
<br/><br/>
Affinity is a score out of 10, representing how much a person's expectations match their partner's personality. <br/>(If you want to go further: affinity is, for each criteria the product of expectations and personality, then summed on criterias, then divided by the sum of expectations which is theoretically 100)

In [11]:
#Ranking the person met
# rate 1-10
ranking_other_person = [
    "attr",
    "sinc",
    "intel",
    "fun",
    "amb",
    "shar"
]

# Define affinity based on one's expectations and the other's rating
def affinity(df, i, expectations, ranking):
    affinity = 0
    expect_sum = 0
    # Evaluating each of the 6 expectations axis one by one
    for j in range(len(expectations)): 
        expect = dating.loc[i, expectations[j]] # importance of this attibute in one's expectations
        affinity += expect * (dating.loc[i, ranking[j]]) # increase affinity with previous line expect coef multiplied by the other's person rating
        expect_sum += expect # sum of expectations point, should be 100 if notations rules are respected
    if expect_sum == 0: return np.nan # this person did not give any expectation
    else: return affinity / expect_sum # affinity ponderated by total expectation score

# Create affinity column
dating["affinity"] = np.nan 
for i in dating.index:
    dating.loc[i, "affinity"] = affinity(dating, i, expectations_index, ranking_other_person)

Visualise some results

In [12]:
# Display a sample of expectations, other person's rating, affinity and waves
display(dating.sample(10)[ranking_other_person+expectations_index+["affinity", "wave"]])

,attr,sinc,intel,fun,amb,shar,attr1_1,sinc1_1,intel1_1,fun1_1,amb1_1,shar1_1,affinity,wave
3819,4.0,10.0,10.0,8.0,10.0,2.0,20.00,20.00,20.00,10.00,20.00,10.00,7.800000,11
3513,8.0,7.0,8.0,7.0,7.0,4.0,18.00,10.00,30.00,10.00,10.00,22.00,6.820000,10
833,6.0,8.0,9.0,5.0,9.0,7.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,3
1227,6.0,7.0,10.0,7.0,10.0,7.0,15.00,15.00,20.00,15.00,20.00,15.00,8.050000,4
7569,8.0,8.0,8.0,8.0,8.0,7.0,58.00,5.00,8.00,10.00,7.00,12.00,7.880000,21
3624,3.0,5.0,5.0,3.0,5.0,2.0,25.00,7.00,25.00,25.00,8.00,10.00,3.700000,11
5422,7.0,8.0,8.0,7.0,7.0,7.0,20.51,14.53,24.79,17.09,5.98,17.09,7.393239,14
6495,5.0,7.0,8.0,5.0,6.0,5.0,16.00,25.00,20.00,12.00,12.00,15.00,6.220000,16
5257,4.0,5.0,6.0,4.0,6.0,NaN,15.00,10.00,25.00,25.00,10.00,15.00,NaN,14
7806,4.0,8.0,9.0,3.0,8.0,2.0,50.00,20.00,10.00,5.00,10.00,5.00,5.550000,21


Now we study the influence of the affinity score on the final decision taken by the person. The person can choose to ask for a second date (say "Yes") or not (say "No")

In [13]:
# Separate each row by one's final decision, then caclculate affinity mean for each group
dating["dec"] = dating["dec"].replace({0: "No", 1:"Yes"})
display(dating.groupby("dec")["affinity"].mean())

dec
No     6.062689
Yes    7.335092
Name: affinity, dtype: float64

In [14]:
# Same but separate men and women first
dating["gender"] = dating["gender"].replace({0: "Female", 1:"Male"})
display(dating.groupby(["gender", "dec"])["affinity"].mean())

gender  dec
Female  No     6.065627
        Yes    7.334928
Male    No     6.059359
        Yes    7.335222
Name: affinity, dtype: float64

<br/>We see that people have a higher affinity with people they say "Yes" to, regardless of their gender. <br/><br/>
Now, lets visualise the volume of "Yes" and "No" decisions depending on the affinity.

In [15]:
# Small function to beautify the graphs with nice labels
def aff_bin(aff):
    if 0 <= aff < 1: return "0-1"
    if  1<= aff < 2: return "1-2"
    if  2<= aff < 3: return "2-3"
    if  3<= aff < 4: return "3-4"
    if  4<= aff < 5: return "4-5"
    if  5<= aff < 6: return "5-6"
    if  6<= aff < 7: return "6-7"
    if  7<= aff < 8: return "7-8"
    if  8<= aff < 9: return "8-9"
    if  9<= aff <= 10: return "9-10"

# Apply beautifying fuction
dating["affinity_bin"] = dating["affinity"].apply(aff_bin)
dating["affinity_bin"]

# Plot number of YES and NO for each affinity segment
#["0-1", "1-2", "2-3", "3-4", "4-5", "5-6", "6-7", "7-8", "8-9", "9-10"]
fig1 = px.histogram(
    dating, 
    x="affinity_bin", 
    color="dec", 
    category_orders={'affinity_bin':["0-1", "1-2", "2-3", "3-4", "4-5", "5-6", "6-7", "7-8", "8-9", "9-10"]},
    title="Influence of affinity on the final decision (volume)"
) 
fig1.show()

To go further we need to study the proportions.

In [16]:
# Calculating percentage from previous graph
yes = dating[dating["dec"] == "Yes"].groupby("affinity_bin").size() # calculate yes volume
no = dating[dating["dec"] == "No"].groupby("affinity_bin").size() # calculate no volume
yes_percentage = (yes / (yes + no)).reset_index() # calculate percentage
yes_percentage.columns = ["affinity_bin", "yes_percentage"] # beautifying

# Plot evolution of "yes" decision accross affinity segments 
fig = go.Figure(
    data = go.Scatter(
        x = yes_percentage["affinity_bin"], 
        y = yes_percentage["yes_percentage"]),
    layout = go.Layout(
        title = go.layout.Title(text = "Chance to ask for a second date based on affinity score (self evaluated)", x = 0.5),
        xaxis = go.layout.XAxis(title = 'Affinity'),
        yaxis = go.layout.YAxis(title = 'Yes percentage')
    )
)
fig.show()

### Conclusion regarding people consistency
There seems to be a correlation between how the other person fits one's criterias and the match chance. People are consistent between what they think they want before the event and the final choice they make.

## 3. Is sharing interests important?
First, lets visualise the volume of "Yes" and "No" decisions depending on the shared interest score. This score is evaluated by the person giving their decision.

In [17]:
# Plot final decision for different shared interests segments
fig1 = px.histogram(dating, x="shar", color="dec", title="Volume of decisions to ask for a second date regarding shared interest score") #nbins=60
fig1.show()

To go further we need to study the proportions.

In [18]:
# Calculating percentage from previous graph
yes = dating[dating["dec"] == "Yes"].groupby("shar").size() # yes volume
yes = pd.DataFrame(yes.drop([5.5, 6.5, 7.5, 8.5])) #droping columns with only one value to clarify plot
no = dating[dating["dec"] == "No"].groupby("shar").size() # no volume
no = pd.DataFrame(no.drop([6.5, 7.5])) #droping columns with only one value to clarify plot
yes_percentage = (yes / (yes + no)).reset_index() # percentage
yes_percentage.columns = ["shared_interests", "yes_percentage"] # beautifying

# Plot evolution of "yes" decision accross shared interests segments 
fig = go.Figure(
    data = go.Scatter(
        x = yes_percentage["shared_interests"], 
        y = yes_percentage["yes_percentage"]),
    layout = go.Layout(
        title = go.layout.Title(text = "Chance to ask for a second date based on shared interests score (self evaluated)", x = 0.5),
        xaxis = go.layout.XAxis(title = 'Shared interests'),
        yaxis = go.layout.YAxis(title = 'Yes chance')
    )
)
fig.show()

Here we see a correlation between the shared interest score and the chance the person will ask for a second date. It should also be the case with matchs (both person saying "Yes").

In [19]:
# Beautifying labels
dating["match"] = dating["match"].replace({0: "No match", 1:"Match"})

In [20]:
# Calculating percentage from previous graph
match = dating[dating["match"] == "Match"].groupby("shar").size()
match = pd.DataFrame(match.drop([6.5, 7.5, 8.5])) #droping columns with only one value to clarify plot
no = dating[dating["match"] == "No match"].groupby("shar").size()
no = pd.DataFrame(no.drop([5.5, 6.5, 7.5, 8.5])) #droping columns with only one value to clarify plot
match_percentage = (match / (match + no)).reset_index()
match_percentage.columns = ["shared_interests", "match_percentage"]

# Plot evolution of "match" (i.e. two matching "yes" final decisions) accross shared interests segments 
fig = go.Figure(
    data = go.Scatter(
        x = match_percentage["shared_interests"], 
        y = match_percentage["match_percentage"]),
    layout = go.Layout(
        title = go.layout.Title(text = "Chance to match based on shared interests score", x = 0.5),
        xaxis = go.layout.XAxis(title = 'Shared interests'),
        yaxis = go.layout.YAxis(title = 'Match percentage')
    )
)
fig.show()

### Conclusion regarding shared interests

Sharing interest seems to have a significant impact on getting a second date. Unfortunately correlation is not causation, and it does not mean that pretending to share an interest will help you get a second date. Further experiments would be necessary to answer this question.